## Import Packages

In [ ]:
# If haven't already downloaded all NLTK
import nltk
nltk.download('all')

In [32]:
# For cosine similarity function
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

# Data storage and manipulation
import pandas as pd

# Graphing
import matplotlib.pyplot as plt

# If running large dataset, used for indicating time through loops
import time

## Functions

In [39]:
def cosine_similarity(x, y, stop_words):
    """
    Takes two sentences (string), x and y, and returns cosin similarity value
    """
    
    # Remove special characters
    x = re.sub("[^a-zA-Z]", " ", x)
    y = re.sub("[^a-zA-Z]", " ", y)
    
    # Tokenize sentences / transform into list of words
    x_list = word_tokenize(x)
    y_list = word_tokenize(y)
    
    # Initialize lists that will be used for vectors
    x_vector = []
    y_vector = []
    
    # Remove stop words from the tokenized lists and get union set
    x_set = {word for word in x_list if not word in stop_words}
    y_set = {word for word in y_list if not word in stop_words}
    union_set = x_set.union(y_set)
    
    # Create vectors
    for word in union_set:
        if word in x_set:
            x_vector.append(1)
        else:
            x_vector.append(0)
        if word in y_set:
            y_vector.append(1)
        else:
            y_vector.append(0)
    
    # Cosine calculation
    c = 0
    for i in range(len(union_set)):
        c += x_vector[i] * y_vector[i]
    cosine_similarity = c / float((sum(x_vector) * sum(y_vector))**0.5)
    
    return cosine_similarity


def get_closest_clusters(cluster_dict, threshold, stop_words):
    """
    Takes cluster dictionary where keys are clusters and values are utterances
    Returns list of two keys representing closest clusters
    """
    # start = time.time()
    
    max_similarity = 0
    combine_clusters = []


    # Iterate through all combinations of clusters using key index so I don't repeat combinations
    for cluster_i in range(len(cluster_dict.keys())):
        for comparison_cluster_i in range(cluster_i+1, len(cluster_dict.keys())):

            # Get actual keys / cluster numbers
            key_1 = list(cluster_dict.keys())[cluster_i]
            key_2 = list(cluster_dict.keys())[comparison_cluster_i]
            

            # If not the same cluster
            if key_1 != key_2:

                # Check similarity of the utteranceclusters using average if the utterance
                # is already clustered.Retain keys of clusters or utterances with max similarity

                # If comparing cluster to utterance
                if (len(cluster_dict[key_1]) > 1) and (len(cluster_dict[key_2]) == 1):
                    sum_sim = 0
                    for utterance in cluster_dict[key_1]:
                        sum_sim += cosine_similarity(utterance, cluster_dict[key_2][0], stop_words)
                    similarity = sum_sim / len(cluster_dict[key_1])
                    if similarity > max_similarity:
                        max_similarity = similarity
                        combine_clusters = [key_1, key_2]

                # If comparing utterance to cluster
                elif (len(cluster_dict[key_1]) == 1) and (len(cluster_dict[key_2]) > 1):
                    sum_sim = 0
                    for utterance in cluster_dict[key_2]:
                        sum_sim += cosine_similarity(cluster_dict[key_1][0], utterance, stop_words)
                    similarity = sum_sim / len(cluster_dict[key_2])
                    if similarity > max_similarity:
                        max_similarity = similarity
                        combine_clusters = [key_1, key_2]

                # If comparing cluster to cluster
                elif (len(cluster_dict[key_1]) > 1) and (len(cluster_dict[key_2]) > 1):
                    sum_sim = 0
                    for utterance_1 in cluster_dict[key_1]:
                        for utterance_2 in cluster_dict[key_2]:
                            sum_sim += cosine_similarity(utterance_1, utterance_2, stop_words)
                    similarity = sum_sim / (len(cluster_dict[key_1]) * len(cluster_dict[key_2]))
                    if similarity > max_similarity:
                        max_similarity = similarity
                        combine_clusters = [key_1, key_2]                

                # If comparing utterance to utterance
                else:
                    similarity = cosine_similarity(cluster_dict[key_1][0], cluster_dict[key_2][0], stop_words)
                    if similarity > max_similarity:
                        max_similarity = similarity
                        combine_clusters = [key_1, key_2]
    
    # Checks to make sure the similarity of the clusters is above the threshold
    if max_similarity < threshold:
        combine_clusters = []

    # end = time.time()
    # print('get_closest_clusters: {}'.format(end-start)) 
        
    return combine_clusters

def update_cluster_dict(combine_clusters, cluster_dict):
    """
    Takes combine_clusters list of two keys representing clusters to be combined and the cluster dictionary
    Adds all values from second key to first key values, deletes second key
    Returns updated cluster dictionary
    """
    
    # start = time.time()
    
    for key, value in cluster_dict.items():
        if key == combine_clusters[1]:
            cluster_dict[combine_clusters[0]].append(cluster_dict[combine_clusters[1]][0])
    del cluster_dict[combine_clusters[1]]    

    # end = time.time()
    # print('update_cluster_dict: {}'.format(end-start))    

    return cluster_dict

def get_wcas(cluster_dict, stop_words):
    """
    Takes cluster dictionary
    Returns within cluster average similarity
    """
    # start = time.time()
    
    wcas = {}
    for key, value in cluster_dict.items():
        if len(value) > 1:
            sum_sim = 0
            d = 0
            for i in range(len(value)):
                for j in range(i+1, len(value)):
                    sum_sim += cosine_similarity(value[i], value[j], stop_words)
                    d += 1
            wcas[key] = sum_sim / d
    
    sum_sim = 0
    for value in wcas.values():
        sum_sim += value
    overall_wcas = sum_sim / len(wcas.values())
    
    # end = time.time()
    # print('get_wcas: {}'.format(end-start))
    
    return overall_wcas, wcas

def utterance_hierarchical_clustering(cluster_dict, threshold, stop_words):
    """
    Takes cluster dictionary and threshold value between 0 and 1 for the lowest cosine similarity
    within any cluster
    Returns cluster dictionary, overall within cluster average similarity list, final overall within cluster
    average similarity, and within cluster average similarity dictionary
    """
    # start_time = time.time()
    
    overall_wcas_list = []
    overall_wcas = 0
    wcas = {}
    num_clusters = len(cluster_dict.keys())
    lowest_similarity = 1

    # Clustering loop runs until there is only 1 cluster, the within cluster average similarity for one of
    # the clusters dips below threshold, or there is 0 similarity between any cluster
    while (num_clusters > 1) and (lowest_similarity > threshold):
               
        combine_clusters = get_closest_clusters(cluster_dict, threshold, stop_words)

        if len(combine_clusters) == 0:
            break

        cluster_dict = update_cluster_dict(combine_clusters, cluster_dict)                    
        overall_wcas, wcas = get_wcas(cluster_dict, stop_words)
        num_clusters = len(cluster_dict.keys())
        overall_wcas_list.append(overall_wcas)
        
        # execution_time = time.time() - start_time
        # start_time = time.time()
        
        # print(execution_time, num_clusters)
        
    return cluster_dict, overall_wcas_list, overall_wcas, wcas

def print_results(cluster_dict, overall_wcas, wcas):
    print('Final Overall WCAS: {} \n'.format(overall_wcas))
    print('Cluster \t WCAS \t \t \t \t Length')
    for k, v in wcas.items():
        print('{} \t \t {} \t \t {}'.format(k, v, len(cluster_dict[k])))
    print('\n')
    print('Cluster \t Utterance')
    for k, v in cluster_dict.items():
        print('{} \t \t {}'.format(k, v))

## Data Preparation

In [47]:
# Read utterances into Pandas df
utterances = pd.read_csv('Validation Data.csv', header=0, names=['Utterance'])

# Put utterances into cluster dictionary where each utterance is it's own cluster
cluster_dict = {}
for i in range(len(utterances.values.tolist())):
    cluster_dict[i] = utterances.values.tolist()[i]

## Hierarchical Clustering

In [48]:
stop_words = stopwords.words('english')

start_time = time.time()
cluster_dict, overall_wcas_list, overall_wcas, wcas = utterance_hierarchical_clustering(cluster_dict, 0.2, stop_words)
print('Time taken: {:.2f} seconds \n'.format(time.time()-start_time))
print('Utterances: {} \n'.format(len(utterances['Utterance'])))
print_results(cluster_dict, overall_wcas, wcas)

Time taken: 27.59 seconds 

Utterances: 69 

Final Overall WCAS: 0.3878871770983733 

Cluster 	 WCAS 	 	 	 	 Length
0 	 	 0.5924132844543113 	 	 4
1 	 	 0.32823858140536927 	 	 4
2 	 	 0.38888888888888884 	 	 3
5 	 	 0.3960108206551019 	 	 4
7 	 	 0.25796287669169626 	 	 4
15 	 	 0.39814239699997195 	 	 4
19 	 	 0.35355339059327373 	 	 2


Cluster 	 Utterance
0 	 	 ["I'm not happy with the service, what do I have to do to submit a consumer complaint?", 'I am not happy with the service, could I file a consumer complaint?', 'I am discontent with the service, how could I file a fucking complaint?', 'im discontent with the service and i want to file a consumer complaint']
1 	 	 ['do you mind asking Alexa how I could check my order, please?', 'please, can you ask Alexa where I could notify problems making a payment?', 'I have an issue making a payment, how can I report it?', 'I have an issue when trying to make a payment with card, can you help me?']
2 	 	 ['ask alexa how to call customer s

*Clusters do not seem distinct. For example, cluster 19 has 2 completely different sentences.*